[`pidgin.weave`](weave.md.ipynb) allows code to be execute with the display function.

In [1]:
    if __name__ == '__main__': 
        %reload_ext pidgin
        #         from pidgin.references import *

In [2]:
    from pidgin import shell

In [3]:
    import builtins, ast, jinja2, sys, nbconvert, nbformat, contextlib, collections, IPython, mistune, doctest, yaml, re, traitlets, textwrap, types, typing as t, importlib, importnb,  jinja2, builtins, jsonschema

In [4]:
    def flatten(str): return '&#xA;'.join(str.splitlines())

In [5]:
    class Template(nbconvert.exporters.TemplateExporter, IPython.core.formatters.DisplayFormatter, shell.Shell):
        def __init__(Template, *args, **kwargs):
            nbconvert.exporters.TemplateExporter.__init__(Template)
            shell.Shell.__init__(Template, *args, **kwargs)
            IPython.core.formatters.DisplayFormatter.__init__(Template, *args, **kwargs)
            Template._create_environment()
            
            
        def _create_environment(Template):
            environment = super()._create_environment()
            environment.undefined = jinja2.StrictUndefined
            environment.finalize = Template.finalize
            return environment
        
        def render(Template, object, **kwargs):
            return Template.environment.from_string(object).render(**{**vars(builtins), **Template.user_ns, **kwargs})
        
        def finalize(Template, object) -> str:
            if isinstance(object, str): 
                try: return ast.parse(str) and object
                except: return Template.user_ns.get(object, object)
            bundle, metadata = Template.format(object)
            for type in [str for str in Template.active_types if str != 'text/plain'] + ['text/plain']:
                if type in bundle: 
                    object = bundle[type]
                    if type.startswith('image') and 'svg' not in type: object = format_images(type, bundle)._repr_html_()
                    if type == 'text/latex': 
                        if object.startswith('$$') and object.endswith('$$'): object = object[1:-1]
                    break
            return object


In [6]:
    def format_bytes(bytes, object):
        return {'text/html': format_images('image/png', {'image/png': bytes}), 'text/plain': repr(object),}, {}

In [7]:
    def format_images(type, bundle):
        str = bundle[type]        
        if isinstance(str, bytes): str = __import__('base64').b64encode(str).decode('utf-8')
            
        if type in ('image/svg+xml', 'text/html'):  ...
        elif str.startswith('http'): str = F"""<img src="{str}"/>"""
        else: str = F"""<img src="data:{type};base64,{str}"/>"""
        return IPython.display.HTML(str)